In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Post-Traumatic_Stress_Disorder"
cohort = "GSE67663"

# Input paths
in_trait_dir = "../../input/GEO/Post-Traumatic_Stress_Disorder"
in_cohort_dir = "../../input/GEO/Post-Traumatic_Stress_Disorder/GSE67663"

# Output paths
out_data_file = "../../output/preprocess/Post-Traumatic_Stress_Disorder/GSE67663.csv"
out_gene_data_file = "../../output/preprocess/Post-Traumatic_Stress_Disorder/gene_data/GSE67663.csv"
out_clinical_data_file = "../../output/preprocess/Post-Traumatic_Stress_Disorder/clinical_data/GSE67663.csv"
json_path = "../../output/preprocess/Post-Traumatic_Stress_Disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profiles in comorbid PTSD and depression"
!Series_summary	"This is a genome-wide differential gene expression survey of cases of comorbid PTSD and depression versus controls"
!Series_overall_design	"We compared genome-wide gene expression profiles of cases of comorbid PTSD and depression to those of controls, adjusting for sex, age, and ancestry via genotypic principal components"
Sample Characteristics Dictionary:
{0: ['Sex: female', 'Sex: male'], 1: ['age: 44', 'age: 74', 'age: 47', 'age: 38', 'age: 37', 'age: 48', 'age: 52', 'age: 56', 'age: 21', 'age: 20', 'age: 54', 'age: 55', 'age: 36', 'age: 31', 'age: 39', 'age: 40', 'age: 57', 'age: 45', 'age: 50', 'age: 24', 'age: 30', 'age: 26', 'age: 43', 'age: 34', 'age: 49', 'age: 19', 'age: 18', 'age: 60', 'age: 27', 'age: 29'], 2: ['comorbid ptsd and depression status: 1', 'comorbid ptsd and depression status: 0'], 3: ['rin: 6.1', 'rin: 6.7', 'rin: 7.3', 'rin: 6.3', 'rin: 7.5', 'rin

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
from typing import Optional, Callable, Dict, Any
import os
import json

# 1. Gene Expression Data Availability
# Based on background info, this dataset contains genome-wide gene expression profiles
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait: Post-Traumatic Stress Disorder
# From the sample characteristics, we can see row 2 has "comorbid ptsd and depression status"
trait_row = 2

# Age: Available in row 1
age_row = 1

# Gender: Available in row 0 (Sex)
gender_row = 0

# 2.2 Data Type Conversion Functions

def convert_trait(value: str) -> Optional[int]:
    """Convert PTSD trait values to binary format (0/1)."""
    if value is None:
        return None
    
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Based on the sample characteristics, "comorbid ptsd and depression status: 1" is case (PTSD)
    # and "comorbid ptsd and depression status: 0" is control (no PTSD)
    if value == '1':
        return 1  # PTSD case
    elif value == '0':
        return 0  # Control
    else:
        return None  # Unknown or invalid value

def convert_age(value: str) -> Optional[float]:
    """Convert age values to continuous format."""
    if value is None:
        return None
    
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender values to binary format (0=female, 1=male)."""
    if value is None:
        return None
    
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if value == 'female':
        return 0
    elif value == 'male':
        return 1
    else:
        return None

# 3. Save Metadata for Initial Filtering
# Determine trait data availability (if trait_row is not None)
is_trait_available = trait_row is not None

# Validate and save cohort info for initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Only if trait_row is not None (i.e., clinical data is available)
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary that was shown in the previous step
    # First, we need to transform the dictionary into a proper format for DataFrame creation
    sample_characteristics = {
        0: ['Sex: female', 'Sex: male'], 
        1: ['age: 44', 'age: 74', 'age: 47', 'age: 38', 'age: 37', 'age: 48', 'age: 52', 'age: 56', 'age: 21', 'age: 20', 'age: 54', 'age: 55', 'age: 36', 'age: 31', 'age: 39', 'age: 40', 'age: 57', 'age: 45', 'age: 50', 'age: 24', 'age: 30', 'age: 26', 'age: 43', 'age: 34', 'age: 49', 'age: 19', 'age: 18', 'age: 60', 'age: 27', 'age: 29'], 
        2: ['comorbid ptsd and depression status: 1', 'comorbid ptsd and depression status: 0']
        # Additional fields from the sample characteristics dictionary are omitted as they're not needed
    }
    
    # Determine the number of samples based on the longest list in the dictionary
    num_samples = max(len(values) for values in sample_characteristics.values())
    
    # Create a DataFrame with one column per row in the sample characteristics
    clinical_data = pd.DataFrame()
    
    # Fill the DataFrame with sample characteristics data
    for row_idx, values in sample_characteristics.items():
        # Create a series with length = number of samples, filled with the values from the list
        # If there are fewer values than samples, the values will be recycled
        clinical_data[row_idx] = pd.Series([values[i % len(values)] for i in range(num_samples)])
    
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_features)
    print("Preview of clinical features:")
    print(preview)
    
    # Save clinical data to output file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical features:
{0: [nan, nan, 0.0], 1: [nan, 74.0, nan], 2: [1.0, 0.0, nan]}
Clinical data saved to ../../output/preprocess/Post-Traumatic_Stress_Disorder/clinical_data/GSE67663.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651262', 'ILMN_1651278', 'ILMN_1651279', 'ILMN_1651315',
       'ILMN_1651316', 'ILMN_1651328', 'ILMN_1651336', 'ILMN_1651346',
       'ILMN_1651347', 'ILMN_1651354', 'ILMN_1651358', 'ILMN_1651364',
       'ILMN_1651373', 'ILMN_1651378', 'ILMN_1651385', 'ILMN_1651403'],
      dtype='object', name='ID')

Gene data dimensions: 13048 genes × 184 samples


### Step 4: Gene Identifier Review

In [5]:
# Analyzing the gene identifiers
# These are Illumina BeadArray identifiers (ILMN_) which are not human gene symbols
# They are probe IDs for the Illumina microarray platform that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1825594', 'ILMN_1810803', 'ILMN_1722532', 'ILMN_1884413', 'ILMN_1906034'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['Unigene', 'RefSeq', 'RefSeq', 'Unigene', 'Unigene'], 'Search_Key': ['ILMN_89282', 'ILMN_35826', 'ILMN_25544', 'ILMN_132331', 'ILMN_105017'], 'Transcript': ['ILMN_89282', 'ILMN_35826', 'ILMN_25544', 'ILMN_132331', 'ILMN_105017'], 'ILMN_Gene': ['HS.388528', 'LOC441782', 'JMJD1A', 'HS.580150', 'HS.540210'], 'Source_Reference_ID': ['Hs.388528', 'XM_497527.2', 'NM_018433.3', 'Hs.580150', 'Hs.540210'], 'RefSeq_ID': [nan, 'XM_497527.2', 'NM_018433.3', nan, nan], 'Unigene_ID': ['Hs.388528', nan, nan, 'Hs.580150', 'Hs.540210'], 'Entrez_Gene_ID': [nan, 441782.0, 55818.0, nan, nan], 'GI': [23525203.0, 89042416.0, 46358420.0, 7376124.0, 5437312.0], 'Accession': ['BU678343', 'XM_497527.2', 'NM_018433.3', 'AW629334', 'AI818233'], 'Symbol': [nan, 'LOC441782', 'JMJD1A', nan, nan], 'Prot

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns in gene_annotation contain probe IDs and gene symbols
# Based on the preview, 'ID' contains probe IDs matching our gene expression data
# 'Symbol' contains gene symbols

# 2. Get the gene mapping dataframe by extracting the relevant columns
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Symbol')

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression data
# The function handles probes that map to multiple genes by dividing expression equally
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Normalize gene symbols in the index for consistency
gene_data = normalize_gene_symbols_in_index(gene_data)

# Print information about the gene data after mapping
print(f"\nAfter mapping, gene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
print("First 10 gene symbols:")
print(gene_data.index[:10])

# Save the gene expression data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")



After mapping, gene data dimensions: 8689 genes × 184 samples
First 10 gene symbols:
Index(['A1BG', 'AACS', 'AAMDC', 'AAMP', 'AAR2', 'AARS1', 'AARSD1', 'AASDH',
       'AASDHPPT', 'AATF'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Post-Traumatic_Stress_Disorder/gene_data/GSE67663.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Gene expression data extraction and mapping (using the results from previous steps)
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_data = get_genetic_data(matrix_file)
gene_annotation = get_gene_annotation(soft_file)

# 2. Gene mapping (since requires_gene_mapping = True from Step 4)
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Symbol')
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# 3. Normalize gene symbols in gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"First 5 normalized gene symbols: {normalized_gene_data.index[:5].tolist() if len(normalized_gene_data) > 0 else 'No genes after normalization'}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 4. Re-extract clinical data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# From Step 2, we identified the correct rows
trait_row = 2  # comorbid ptsd and depression status
age_row = 1    # age row
gender_row = 0  # gender row

def convert_trait(value):
    """Convert PTSD trait values to binary format (0/1)."""
    if value is None:
        return None
    
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Based on the sample characteristics, "comorbid ptsd and depression status: 1" is case (PTSD)
    # and "comorbid ptsd and depression status: 0" is control (no PTSD)
    if value == '1':
        return 1  # PTSD case
    elif value == '0':
        return 0  # Control
    else:
        return None  # Unknown or invalid value

def convert_age(value):
    """Convert age values to continuous format."""
    if value is None:
        return None
    
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)  # Keep age values positive
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender values to binary format (0=female, 1=male)."""
    if value is None:
        return None
    
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if value == 'female':
        return 0
    elif value == 'male':
        return 1
    else:
        return None

# Extract clinical features with correct row indices and conversion functions
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Debug: Show preview of clinical data
print("Preview of clinical data:")
print(preview_df(selected_clinical_df))

# Save clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 5. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 6. Handle missing values
linked_data = handle_missing_values(linked_data, trait_col=trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 7. Determine if trait is biased
print("\nChecking for bias in the trait variable:")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 8. Conduct final quality validation
is_trait_available = True  # We confirmed trait data is available
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset studies gene expression in comorbid PTSD and depression versus controls."
)

# 9. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for trait association studies, linked data not saved.")

Gene data shape after normalization: (8689, 184)
First 5 normalized gene symbols: ['A1BG', 'AACS', 'AAMDC', 'AAMP', 'AAR2']


Normalized gene data saved to ../../output/preprocess/Post-Traumatic_Stress_Disorder/gene_data/GSE67663.csv
Preview of clinical data:
{'GSM1652966': [1.0, 44.0, 0.0], 'GSM1652967': [0.0, 74.0, 0.0], 'GSM1652968': [1.0, 47.0, 1.0], 'GSM1652969': [1.0, 38.0, 0.0], 'GSM1652970': [1.0, 37.0, 0.0], 'GSM1652971': [0.0, 48.0, 0.0], 'GSM1652972': [0.0, 52.0, 1.0], 'GSM1652973': [1.0, 56.0, 0.0], 'GSM1652974': [0.0, 21.0, 0.0], 'GSM1652975': [0.0, 20.0, 0.0], 'GSM1652976': [0.0, 54.0, 0.0], 'GSM1652977': [1.0, 55.0, 1.0], 'GSM1652978': [0.0, 56.0, 0.0], 'GSM1652979': [0.0, 54.0, 0.0], 'GSM1652980': [1.0, 47.0, 1.0], 'GSM1652981': [0.0, 36.0, 0.0], 'GSM1652982': [0.0, 56.0, 0.0], 'GSM1652983': [0.0, 44.0, 1.0], 'GSM1652984': [1.0, 31.0, 0.0], 'GSM1652985': [1.0, 39.0, 0.0], 'GSM1652986': [1.0, 40.0, 0.0], 'GSM1652987': [1.0, 57.0, 1.0], 'GSM1652988': [1.0, 39.0, 0.0], 'GSM1652989': [1.0, 47.0, 1.0], 'GSM1652990': [1.0, 39.0, 1.0], 'GSM1652991': [1.0, 45.0, 0.0], 'GSM1652992': [1.0, 50.0, 0.0], '

Data shape after handling missing values: (184, 8692)

Checking for bias in the trait variable:
For the feature 'Post-Traumatic_Stress_Disorder', the least common label is '0.0' with 72 occurrences. This represents 39.13% of the dataset.
The distribution of the feature 'Post-Traumatic_Stress_Disorder' in this dataset is fine.

Quartiles for 'Age':
  25%: 32.75
  50% (Median): 45.0
  75%: 52.0
Min: 18.0
Max: 74.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 46 occurrences. This represents 25.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Post-Traumatic_Stress_Disorder/GSE67663.csv
